In [1]:
import serial
import serial.tools.list_ports
import time
import os
import cv2
import csv
import numpy as np
from datetime import datetime


In [2]:
# BAUD_RATE = 9600

In [3]:
# def connect():
#     # connect to Arduino
#     ports = list(serial.tools.list_ports.comports())  # Create a list with info for all devices
#     connected = False
#     # Look for the word 'Arduino' in device info, store this port
#     for p in ports:
#         if "Arduino" in p[1]:
#             connected = True
#             port_arduino = p[0]
#             print("Connected to Arduino on port: ", port_arduino)
#             break
#     if not connected:
#         print("No Arduino Found")

#     ser = serial.Serial(port_arduino, BAUD_RATE)
#     return ser

In [5]:
# BAUD_RATE = 9600
# ser = serial.Serial('/dev/ttyACM1', BAUD_RATE)

# def connect():
#     # connect to Arduino
#     ports = list(serial.tools.list_ports.comports())  # Create a list with info for all devices
#     connected = False
#     # Look for the word 'Arduino' in device info, store this port
#     for p in ports:
#         if "Arduino" in p[1]:
#             connected = True
#             port_arduino = p[0]
#             print("Connected to Arduino on port: ", port_arduino)
#             break
#     if not connected:
#         print("No Arduino Found")

#     ser = serial.Serial(port_arduino, BAUD_RATE)
#     return ser


# def send_cmd(ser,cmd):
#     """
#     send string command to Arduino

#     :param cmd: (string) command to send
#     :return:
#     """
#     # print(cmd)
#     for c in cmd:
#         # self.ser.write(c.encode())
#         ser.write(c.encode())
#         time.sleep(.01)
#     ser.write('\r'.encode())
#     time.sleep(.01)

# def clear_buff(ser):
#     while ser.in_waiting:
#         ser.read()

# def read_char(ser):
#     # use Serial.print() on Arduino
#     c = ser.read().decode()
#     return c


In [4]:
# ser = connect()

Connected to Arduino on port:  /dev/ttyACM1


In [7]:
# clear_buff(ser)
# send_cmd(ser, ' ')
# ser.write(b' \n') 
# clear_buff(ser)

In [10]:
# send_cmd(ser, '2 180 0')

In [2]:
import datetime
def get_datetime_str():
    """

    :return: returns current datetime as a string with format yyyy-mm-dd-HH-MM-SS-UUUUUU
    """
    date_time = datetime.datetime.now()
    time_str = str(date_time.time()).split('.') # Get string with just hh:mm:ss
    time_str = time_str[0] + ':'+time_str[1]
    time_str = time_str.replace(':', '-')  # Now hh-mm-ss
    date_str = str(date_time.date())  # Date in format yyyy-mm-dd
    date_time_str = '-'.join([date_str, time_str])
    return date_time_str

In [4]:
def coordinates_new(cam_no):
    camera_matrix = np.array([[682.694, 0, 573.5379], [0, 675.82, 367.535], [0, 0, 1]])
    dist_coeffs = np.array([-0.119, 0.1449, 0.0019, -0.00689, -0.1701])

    aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
    marker_size = 2.6  # or 2.61

        # Function to calculate marker pose
    def estimate_marker_pose(marker_corners):
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(marker_corners, marker_size, camera_matrix, dist_coeffs)
        return rvecs, tvecs
    cap = cv2.VideoCapture(cam_no)
    output_file = 'Randon_testing_data.csv'
    with open(output_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        ret, frame = cap.read()  # Capture a single frame
        cap.release()  # Release the capture device

        if ret:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            corners, ids, _ = cv2.aruco.detectMarkers(gray, aruco_dict)

            if ids is not None:
                rvecs, tvecs = estimate_marker_pose(corners)
                values = {}
                for i in range(len(ids)):
                    cv2.aruco.drawDetectedMarkers(frame, corners)
                    cv2.aruco.drawAxis(frame, camera_matrix, dist_coeffs, rvecs[i], tvecs[i], marker_size * 0.5)
                    # x, y, z = tvecs[0][0] - tvecs[1][0]
                    if ids[i][0] == 3: # that is the robot wrt top cam
                        variable_name = "var_0"
                        values[variable_name] = tvecs[i][0]
                    elif ids[i][0] == 4:
                        variable_name = "var_1"
                        values[variable_name] = tvecs[i][0]
                    # variable_name = "var_" + str(i)
                    # values[variable_name] = tvecs[i][0]
                    print("Marker ID:", ids[i][0])
                    # print("Position (x, y):", x, y)
                print(values)
                if len(values) == 2:
                    x, y, z = values['var_0'] - values['var_1'] #check obj wrt R is (what minus what)
                else:
                    x = y = z = float("NaN")
                timestamp = get_datetime_str()
                writer.writerow([timestamp, x, y])

            cv2.imshow("ArUco Marker Detection", frame)
        # cv2.waitKey(0)

    cv2.destroyAllWindows()

In [12]:
coordinates_new(14)

Marker ID: 4
Marker ID: 3
{'var_1': array([-15.30554252,  11.64165526,  52.1180117 ]), 'var_0': array([-13.02163241, -13.5077895 ,  38.98487807])}


[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (1100) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


# Note: Manually add direction (left/right) and type (contact/ non-contact) in the csv file, and note down the serial number.
# Save the image manually with the same serial no. name for testing convenience

In [9]:
import cv2
from cv2 import aruco
def show_marker():
    # ### Overlay marker ID and video ###
    dict_aruco = aruco.Dictionary_get(aruco.DICT_4X4_50)
    parameters = aruco.DetectorParameters_create()

    cap = cv2.VideoCapture(6)

    try:
        while True:
            ret, frame = cap.read()
            gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

            corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, dict_aruco, parameters=parameters)

            frame_markers = aruco.drawDetectedMarkers(frame.copy(), corners, ids)
            cv2.imshow('frame', frame_markers)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cv2.destroyWindow('frame')
        cap.release()
    except KeyboardInterrupt:
        cv2.destroyWindow('frame')
        cap.release()


In [ ]:
show_marker()

In [ ]:
# 14 = top cam,
# 12 = Front cam #
# 8  = interior cam